In [0]:
import numpy as np 
import pandas as pd
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Model, Input, Sequential
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Bidirectional, Conv1D, GRU
from sklearn.model_selection import train_test_split, StratifiedKFold
from keras.metrics import categorical_accuracy
from keras.regularizers import l1_l2, l2
from keras import backend as K
import tensorflow as tf

In [0]:
import csv
import re
import json
import numpy as np

years = list(range(2017, 2018))

team_map = {'ATL' : 'Atlanta Hawks', 'BOS': 'Boston Celtics', 'BRK': 'Brooklyn Nets', 
			'CHO': 'Charlotte Hornets', 'CHI': 'Chicago Bulls', 'CLE': 'Cleveland Cavaliers',
			'DET': 'Detroit Pistons', 'IND': 'Indiana Pacers', 'MIA': 'Miami Heat',
			'MIL': 'Milwaukee Bucks', 'NYK': 'New York Knicks', 'ORL': 'Orlando Magic',
			'PHI': 'Philadelphia 76ers', 'TOR': 'Toronto Raptors', 'WAS': 'Washington Wizards',
			'DAL': 'Dallas Mavericks', 'DEN': 'Denver Nuggets', 'GSW': 'Golden State Warriors',
			'HOU': 'Houston Rockets', 'LAC': 'Los Angeles Clippers', 'LAL': 'Los Angeles Lakers',
			'MEM': 'Memphis Grizzlies', 'MIN': 'Minnesota Timberwolves', 'NOP': 'New Orleans Pelicans',
			'OKC': 'Oklahoma City Thunder', 'PHO': 'Phoenix Suns', 'POR': 'Portland Trail Blazers',
			'SAC': 'Sacramento Kings', 'SAS': 'San Antonio Spurs', 'UTA': 'Utah Jazz',
			'NJN': 'New Jersey Nets', 'SEA': 'Seattle SuperSonics', 'CHA': 'Charlotte Bobcats'}

stat_encoding = {'fg': 1, 'fg_pct': 2, 'fg3': 3, 'fg3_pct': 4, 'ft': 5, 'ft_pct': 6, 'pts': 7, 'orb': 8,
                'trb': 9, 'ast': 10, 'stl': 11, 'blk': 12, 'tov': 13}

In [0]:
max_count = 10
def get_tuples():
    game_data = []
    for year in years:
        print(year)
        filename = '{}_labeled_events_sentences.csv'.format(year)
        with open(filename) as inFile:
            csv_reader = csv.reader(inFile, delimiter=',')
            next(csv_reader)
            for line in csv_reader:
                event = line[2]
                values = re.findall(r"'(.*?)'", event)
                game_data.append(values)

    return game_data

def one_hot_int(name_num, stat_num, name_enc, stat_enc):
    one_hot = np.zeros(len(name_enc) + len(stat_enc)+1)
    one_hot[name_num] = 1
    one_hot[(len(name_enc) - 1) + stat_num] = 1
  
    return one_hot

def get_encodings(events, vocab_size):
    with open('players_2017.json') as df:
        data = json.load(df)

    encodings = np.zeros((len(events), max_count  * (len(data) + len(stat_encoding)+ 1)))
    print(encodings.shape)
    for j in range(len(events)):
        event = events[j]
        end_point = min(len(event), max_count*3)
        for i in range(0,end_point,3):
            name = event[i]
            stat = event[i+1]
            if name in data.keys() and stat in stat_encoding.keys():
                value = float(event[i+2])
                name_encode = data[name]
                stat_encode = stat_encoding[stat]

                one_hot = one_hot_int(name_encode, stat_encode, data, stat_encoding)
                one_hot[len(one_hot) - 1] = value
               
                encodings[j][i//3*vocab_size: i//3*vocab_size + len(one_hot)] = one_hot


    return encodings, max_count

In [6]:
num_articles = 1000
with open('players_2017.json') as df:
    data = json.load(df)

vocab_size = len(data) + len(stat_encoding) + 1

events = get_tuples()
encodings, max_count = get_encodings(events[:num_articles], vocab_size)
print(vocab_size)
print(max_count)


2017
(1000, 5770)
577
10


In [0]:
encodings[:,576]

In [0]:
def str_to_tup(string):
	game_data = []
	tokens = string.split(')')[:-1]
	for token in tokens:
		values = re.findall(r"'(.*?)'", token)
		game_data.append(values)

	return game_data

train_event_sentences = pd.read_csv('2017_labeled_events_sentences.csv')
train_articles = []
train_events = []
train_event_sentences_sample = train_event_sentences[:num_articles]
for index, row in train_event_sentences_sample.iterrows():
    train_events.append(str_to_tup(row['event']))
    train_articles.append(row['sentence'])

    

In [8]:
import re
article_size = 100
tokenized_articles = [] 
for article in train_articles:
    article = article.replace(',', ' ,').replace('.', ' . ')
    tokens = article.split(' ')
    i = 0
    while i<len(tokens):
        if len(tokens[i]) == 0:
            del tokens[i]
        else:
            i += 1
    end_point = min(article_size - 1, len(tokens))
    curr_article = []
    for i in range(end_point):
        curr_article.append(tokens[i])
    curr_article.append('_STOP_')
    for i in range(end_point + 1, article_size):
        curr_article.append('_PAD_')
    tokenized_articles.append(curr_article)
print(tokenized_articles[5])

['Fournier', 'made', 'three', 'clutch', 'shots', 'to', 'slow', 'the', 'hard', 'charging', 'Miami', 'Heat', ',', 'and', 'the', 'Magic', 'held', 'on', 'for', 'a', '116', '109', 'win', 'on', 'Wednesday', 'night', 'its', 'first', 'season', 'opening', 'victory', 'since', '2012', '.', 'Kelly', 'Olynyk', 'had', '10', 'points', 'in', 'his', 'first', 'game', 'with', 'the', 'Heat', 'and', 'Waiters', 'had', '15', '.', 'But', 'when', 'Miami', 'pulled', 'within', 'two', 'points', 'with', '2:39', 'remaining', ',', 'Fournier', 'went', 'to', 'work', ',', 'scoring', 'seven', 'of', 'his', 'team', 'high', '23', 'points', 'in', 'the', 'final', '2:27', '.', 'James', 'Johnson', ',', 'Dion', 'Waiters', 'and', 'Dragic', 'had', 'been', 'pushing', 'Miami', 'hard', 'while', 'Orlando', 'missed', 'shots', 'during', 'the', '11', '_STOP_']


In [9]:
tokenizer_decoder = Tokenizer(char_level = False, filters= '', num_words=5000, oov_token='_RARE_')
tokenizer_decoder.fit_on_texts(tokenized_articles)
num_words = len(tokenizer_decoder.word_index) + 1
y = tokenizer_decoder.texts_to_sequences(tokenized_articles)
y_final = np.zeros((len(y), article_size, num_words))
print(y_final.shape)
for i in range(len(y)):

    for j in range(len(y[i])):
        
        y_final[i][j][y[i][j]] = 1.0


(1000, 100, 4132)


In [0]:
X = np.asarray(encodings)
X.shape

(1000, 5770)

In [10]:
X = np.asarray(encodings)
X_final = np.zeros((num_articles, article_size, vocab_size * max_count))
print(X_final.shape)
for i in range(num_articles):
    for j in range(article_size):
        X_final[i][j] = X[i]


(1000, 100, 5770)


In [0]:
X[0] == X_final[0][5]

array([ True,  True,  True, ...,  True,  True,  True])

In [11]:
model = Sequential()
model.add(LSTM(500, input_shape=(100, 5770), return_sequences=True))
model.add(LSTM(500, return_sequences=True))
model.add(Dense(4132, activation="softmax"))
# Compile & run training
# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
model.summary()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 500)          12542000  
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 500)          2002000   
_________________________________________________________________
dense_1 (Dense)              (None, 100, 4132)         2070132   
Total params: 16,614,132
Trainable params: 16,614,132
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(X_final, y_final,
          batch_size=50,
          epochs=10,
          validation_split=0.1)

Train on 900 samples, validate on 100 samples
Epoch 1/10
900/900 [==============================] - 27s 30ms/step - loss: 6.3035 - acc: 0.0494 - val_loss: 6.0834 - val_acc: 0.0540
Epoch 2/10
900/900 [==============================] - 22s 25ms/step - loss: 5.9934 - acc: 0.0507 - val_loss: 6.0865 - val_acc: 0.0540
Epoch 3/10
900/900 [==============================] - 22s 25ms/step - loss: 5.9860 - acc: 0.0520 - val_loss: 6.0956 - val_acc: 0.0540
Epoch 4/10
900/900 [==============================] - 22s 25ms/step - loss: 5.9667 - acc: 0.0538 - val_loss: 6.1111 - val_acc: 0.0550
Epoch 5/10
900/900 [==============================] - 22s 25ms/step - loss: 5.9480 - acc: 0.0557 - val_loss: 6.1074 - val_acc: 0.0577
Epoch 6/10
900/900 [==============================] - 22s 25ms/step - loss: 5.9236 - acc: 0.0560 - val_loss: 6.1224 - val_acc: 0.0554
Epoch 7/10
900/900 [==============================] - 22s 25ms/step - loss: 5.8974 - acc: 0.0561 - val_loss: 6.1289 - val_acc: 0.0577
Epoch 8/10
900/9

In [22]:
prediction = model.predict(X_final[1].reshape((1, 100, 5770)))
pred_words = [np.argmax(i) for i in prediction[0]]

index_word = {v: k for k, v in tokenizer_decoder.word_index.items()} 
article_pred = [index_word[i] for i in pred_words]
print(article_pred)

['the', 'had', 'scored', 'points', 'points', 'points', 'points', 'points', 'and', 'and', 'and', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the']
